In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('hw3.ok')

# Homework 3: SQL

## Due Date: Monday, Feb 10th, 11:59PM

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** below.

**Collaborators**: *list  collaborators here*

## Grading

Grading is broken down into autograded answers and free response. 

For autograded answers, the results of your code are compared to provided and/or hidden tests. **For autograded probability questions, the provided tests will only check that your answer is within a reasonable range.**

### Please make sure to save regularly; do not rely on autosave to save your work for you!

**Caution:** Be careful with large SQL queries.  You may need to reboot your Jupyter Hub instance if it stops responding.

## Scoring Breakdown

|Question|Points|
|---|---|
|Q1 | 2 |
|Q2 | 2 |
|Q3 | 1 |
|Q4 | 2 |
|Q5 | 3 |
|Q6 | 3 |
|Q7 | 2 |
|Q8 | 4 |
|Q9 | 4 |


## Introduction

In this homework, you will have a chance to test your SQL knowledge by writing some SQL queries. We're going to be exploring the publicly available [IMDB](https://www.imdb.com) datasets with data about films and the people involved in their creation.

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
import matplotlib.pyplot as plt
import plotly.offline as py
import cufflinks as cf

## Setup

We have constructed an IMDB database snapshot from [IMDB data](https://www.imdb.com/interfaces/).  This is a very large file so we have mounted it in a shared filesystem on Jupyter Hub.  

We will use `run_sql` function in `ds100_utils`.  This allows us to update the database query code should we run into performance issues on the jupyter hub.

In [3]:
from ds100_utils import run_sql
help(run_sql)

## Getting to Know Your Data

Now that we have downloaded our SQLite database, we want to make sure we understand the **schema** of our data before we start writing queries. The following query returns the tables stored in the database.

In [ ]:
for _, row in run_sql("SELECT sql FROM sqlite_master WHERE type='table' ").iterrows():
    print(row['sql'], '\n')

We have four tables.  Notice that the `names` and `titles` tables have primary keys that the `roles` and `ratings` tables reference. 

Documentation for the schema can be found [here](https://www.imdb.com/interfaces/). Note we have renamed `principals` to `roles`.  We have also slightly shorted or simplified several of the column names.

To get a better understanding of the values we will look at 10 records from each table using the `run_sql` command.  

In [ ]:
run_sql("""
    SELECT * FROM names LIMIT 10
""")

In [ ]:
run_sql("""
    SELECT * FROM titles LIMIT 10
""")

In [ ]:
run_sql("""
    SELECT * FROM roles LIMIT 10
""")

In [ ]:
run_sql("""
    SELECT * FROM ratings LIMIT 10
""")

### Important Schema Information

* `names` contains information about individuals involved in films.
* `titles` contains information about the films.
* `roles` relates individuals to titles
* `ratings` contains information about the ratings associated with each film. 
* `names.nid` is a Primary Key
* `titles.tid` is a Primary Key
* `roles.nid` and `roles.tid` are foreign keys that point to `names.nid` and `titles.tid`, respectively.



---

## Question 1

How far back does our data go? Does it only include recent data, or do we have information about older movies and movie stars as well? 

List the **10 oldest films** by `start_year` and then `title` both in **ascending** order.  Do not include films where the `start_year` is `NULL`.  The output should contain the `start_year`, `type`, and `title`.


<!--
BEGIN QUESTION
name: q1
manual: false
points: 2
-->

In [ ]:
oldestMovieQuery = """..."""
oldestMovieDf = run_sql(oldestMovieQuery)
oldestMovieDf

In [ ]:
ok.grade("q1");

---

## Question 2

Next, let's calculate the distribution of films by year. Write a query that returns the **total** films for each `start_year` in the `titles` table as `total`.  Keep in mind that some entries may not have a `start_year` year listed -- you should filter those out.  Order your final results by the `start_year`.

The first 5 lines of the table should look like the following (but you should compute the entire table).
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>start_year</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1894</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1897</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1898</td>
      <td>26</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1899</td>
      <td>12</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1900</td>
      <td>14</td>
    </tr>
  </tbody>
</table>


<!--
BEGIN QUESTION
name: q2
manual: false
points: 2
-->

In [ ]:
yearDistQuery = """..."""
yearDistDf = run_sql(yearDistQuery)
yearDistDf

In [ ]:
ok.grade("q2");

The following should generate an interesting plot of the number of films that premiered each year. Notice there is a dip between the 1920s and late 1940s. Why might that be? *This question is rhetorical; you do not need to write your answer anywhere.*

In [ ]:
yearDistDf.iplot(kind="bar", x="start_year", y="total", 
                 xTitle="Start Year", yTitle="Count", asFigure=True)

---

## Question 3

What are the different kinds of `types` included in the `titles` table? Write a query to find out all the unique `types` of films using the `DISTINCT` keyword.  (**You may not use `GROUP BY`.**)

<!--
BEGIN QUESTION
name: q3
manual: false
points: 1
-->

In [ ]:
typeQuery = """..."""
typeDf = run_sql(typeQuery)
typeDf

In [ ]:
ok.grade("q3");

---

## Question 4

Before we proceed we want to get a better picture of the kinds of jobs that exist.  To do this examine the `roles` table by computing the number of records with each job `category`.  Present the results in descending order by the total counts.

The top of your table should look like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>category</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>actor</td>
      <td>8739100</td>
    </tr>
    <tr>
      <th>1</th>
      <td>actress</td>
      <td>6541392</td>
    </tr>
    <tr>
      <th>2</th>
      <td>self</td>
      <td>6334197</td>
    </tr>
  </tbody>
</table>

However, there should be more rows.


<!--
BEGIN QUESTION
name: q4
manual: false
points: 2
-->

In [ ]:
roleCategoriesQuery = """..."""
roleCategoriesDf = run_sql(roleCategoriesQuery)
roleCategoriesDf

In [ ]:
ok.grade("q4");

If we computed the results correctly we should see a nice horizontal bar chart of the counts per category below:

In [ ]:
roleCategoriesDf.iplot(kind="barh", x ="category", y = "total", xTitle="Count", asFigure=True)

---


## Question 5

Now that we have a better sense of the basics of our data, we can ask some more interesting questions.  In this question, we will explore which 10 **actors** or **actresses** were in the most **movies**.  Construct a table consisting of their name (as `name`) and the number of movies in which they performed as `total` and order the results by the total in descending order and break ties by ordering by `name` in ascending order.

***The query should take < 2 minutes to run.***

*Hint: Google the top of the list and see if it makes sense.*

*Hint: If you want to include a non-aggregate field in the* `SELECT` *clause, it must also be included in the* `GROUP BY` *clause.*

Your result should look something like the following but without `????`:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>????</td>
      <td>809</td>
    </tr>
    <tr>
      <th>1</th>
      <td>????</td>
      <td>584</td>
    </tr>
    <tr>
      <th>2</th>
      <td>????</td>
      <td>565</td>
    </tr>
    <tr>
      <th>3</th>
      <td>????</td>
      <td>507</td>
    </tr>
    <tr>
      <th>4</th>
      <td>????</td>
      <td>411</td>
    </tr>
    <tr>
      <th>5</th>
      <td>????</td>
      <td>409</td>
    </tr>
    <tr>
      <th>6</th>
      <td>????</td>
      <td>391</td>
    </tr>
    <tr>
      <th>7</th>
      <td>????</td>
      <td>388</td>
    </tr>
    <tr>
      <th>8</th>
      <td>????</td>
      <td>382</td>
    </tr>
    <tr>
      <th>9</th>
      <td>????</td>
      <td>360</td>
    </tr>
  </tbody>
</table>


<!--
BEGIN QUESTION
name: q5
manual: false
points: 3
-->

In [ ]:
prolificPerformersQuery = """..."""
prolificPerformersDf = run_sql(prolificPerformersQuery)
prolificPerformersDf

In [ ]:
ok.grade("q5");

---

## Question 6


We also have average `ratings` data consisting of the `num_votes` and the `average_rating` for each title.  Let's first see how many **movies** have ratings.  Construct the following table:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>has_rating</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>no</td>
      <td>????</td>
    </tr>
    <tr>
      <th>1</th>
      <td>yes</td>
      <td>????</td>
    </tr>
  </tbody>
</table>

Where `????` is replaced with the correct values. The row with `no` should have the count for many movies **do not** have ratings, and the row with `yes` should have the count of how many movies **do** have ratings.

You will need to use a `LEFT JOIN`. You may also consider using a `CASE WHEN ... IS NOT NULL THEN 'yes" ... ELSE ... END` statement. `CASE` statements are the SQL-equivalent of Python `if... elif... else` statements. Take a look at the following links for help with `CASE` statements:
- https://mode.com/sql-tutorial/sql-case/
- https://www.w3schools.com/sql/sql_ref_case.asp


<!--
BEGIN QUESTION
name: q6
manual: false
points: 3
-->

In [ ]:
numberNotRatedQuery = """..."""
missingRatingsDf = run_sql(missingRatingsQuery)
missingRatingsDf

In [ ]:
ok.grade("q6");

---

## Question 7


We also have average `ratings` data consisting of the `num_votes` and the `average_rating` for each title.  Which films have the most ratings?  Make a table consisting of the `title`, `num_votes`, and `avg_rating` for the 10 films that have the highest number of ratings.  Sort the result in descending order by the number of votes.


<!--
BEGIN QUESTION
name: q7
manual: false
points: 2
-->

In [ ]:
popularVotesQuery = """..."""
popularVotesDf = run_sql(popularVotesQuery)
popularVotesDf

In [ ]:
ok.grade("q7");

---

## Question 8


How does film length relate to ratings?  To answer this question we want to bin films by length and compute the average of the average ratings within each length bin. We will group movies by 10-minute increments -- that is, one bin for movies \[0, 10) minutes long, another for \[10, 20) minutes, another for \[20, 30) minutes, and so on. Use the following code snippet to help construct 10-minute bins: 

```
ROUND(runtimeMinutes / 10.0 + 0.5) * 10 AS runtime_bin
```

Construct a table containing the **runtime_bin**, the **average** of the **average ratings** (as `avg_rating`), the **average number of votes** (as `avg_num_votes`), and the number of `titles` in that **runtime_bin** (as `total`).  Only include movies with **at least 10000 votes**.  Order the final results by the value of `runtime_bin`.



<!--
BEGIN QUESTION
name: q8
manual: false
points: 4
-->

In [ ]:
runtimeRatingsQuery = """..."""
runtimeRatingsDf = run_sql(runtimeRatingsQuery)
runtimeRatingsDf

In [ ]:
ok.grade("q8");

If your SQL query is correct you should get some intesting plots below.  This might explain why directors keep going a particular direction with film lengths.  

In [ ]:
runtimeRatingsDf.iplot(x="runtime_bin", y="avg_rating", asFigure=True)

In [ ]:
runtimeRatingsDf.iplot(x="runtime_bin", y="avg_num_votes", asFigure=True)

In [ ]:
runtimeRatingsDf.iplot(kind="bar", x="runtime_bin", y="total", asFigure=True)

---

## Question 9


Which **actors** and **actresses** have the highest average ratings across all the **movies** in which they star?  

Construct a table consisting of the **actor's/actress' name**  (as `name`) and their **average_rating** (as `avg_rating`) computed by rescaling their individual performance ratings:

$$
\frac{\sum_m \text{avg_rating}[m] * \text{num_votes}[m]}{\sum_m \text{num_votes}[m]}
$$

Note that if an actor/actress has multiple `role` listings for a film then that film will have a bigger impact in the overall average (this is desired).

Only consider ratings where there are **at least 1000** votes and only consider actors/actresses that have **at least 20 rated performances**. Present the actors/actresses with the top 10 `avg_rating` in descending order and break ties using the actor's/actress' name.


The results should look something like this but without the `????`:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>avg_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>????</td>
      <td>8.477808</td>
    </tr>
    <tr>
      <th>1</th>
      <td>????</td>
      <td>8.410496</td>
    </tr>
    <tr>
      <th>2</th>
      <td>????</td>
      <td>8.396275</td>
    </tr>
    <tr>
      <th>3</th>
      <td>????</td>
      <td>8.365583</td>
    </tr>
    <tr>
      <th>4</th>
      <td>????</td>
      <td>8.355513</td>
    </tr>
    <tr>
      <th>5</th>
      <td>????</td>
      <td>8.352385</td>
    </tr>
    <tr>
      <th>6</th>
      <td>????</td>
      <td>8.333354</td>
    </tr>
    <tr>
      <th>7</th>
      <td>????</td>
      <td>8.330661</td>
    </tr>
    <tr>
      <th>8</th>
      <td>????</td>
      <td>8.269803</td>
    </tr>
    <tr>
      <th>9</th>
      <td>????</td>
      <td>8.265962</td>
    </tr>
  </tbody>
</table>


<!--
BEGIN QUESTION
name: q9
manual: false
points: 4
-->

In [ ]:
topRatedPerformerQuery = """..."""
topRatedPerformerDf = run_sql(topRatedPerformerQuery)
topRatedPerformerDf

In [ ]:
ok.grade("q9");

## THE END!

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

In [ ]:
# Save your notebook first, then run this cell to submit.
ok.submit()